In [1]:
# set workspace
WhoIsWorking = "IA"
WorkLocation = "School"

if WhoIsWorking == "RT":
    if WorkLocation == "Home":
        folder_path = r'C:\Users\rentr\Desktop\ENGO\DataMining\ENGO645_TermProject_Data'
    elif WorkLocation == "School":
        folder_path = r'D:\ENGO645_TermProject_Data'
elif WhoIsWorking == "AA":
    folder_path = ''
elif WhoIsWorking == "IA":
    folder_path = r'C:\Users\afoam\assignments\ENGO645_TorontoCrimeAnalysis'
elif WhoIsWorking == "AI":
    folder_path = r'C:\Adewale_Directory\Capacity_Building\MGIS\Lecture_Note\WINTER2024\ENGO645\Course_Project\Project_Data'

print(folder_path)

C:\Users\afoam\assignments\ENGO645_TorontoCrimeAnalysis


In [ ]:
pip install pysal

In [ ]:
#Install and import neccessary libraries
import os
import numpy as np
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from pysal.explore import esda
from pysal.lib import 
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Load and read the MCI file
MCI_file = os.path.join(folder_path, "MCI_Cleaned.csv") # Update to the actual file location
MCI_data = pd.read_csv(MCI_file)
 
MCI_data #display dataframe

In [ ]:
# Find the count of offenses by neighborhood to aggregate data to Neighbourhood level
neighbourhood_counts = MCI_data['NEIGHBOURHOOD_158'].value_counts().reset_index()
neighbourhood_counts.columns = ['NeighbourhoodName', 'Crime_Count']  # Rename columns

neighbourhood_counts

In [5]:
# Plot the Crime Count Histogram
neighbourhood_counts['Crime_Count'].plot(kind='hist', bins=20, title='Crime_Count')
plt.gca().spines[['top', 'right',]].set_visible(False)

In [6]:
# Check normality test using Shapiro-Wilk
from scipy.stats import shapiro

# Perform Shapiro-Wilk test
stat, p_value = shapiro(neighbourhood_counts['Crime_Count'])

# Print the results
print('Shapiro-Wilk Test Statistic:', stat)
print('p-value:', p_value)

In [ ]:
# Load and read the Normalized Neighbourhood file
neighbourhood_file = os.path.join(folder_path, "Neighbourhood_Cleaned_Normalized.csv")
neighbourhood_data = pd.read_csv(neighbourhood_file)

In [ ]:
# Check Neighbourhood File for 'geometry' column
if 'geometry' in neighbourhood_data.columns:
    neighbourhood_data.drop(columns=['geometry'], inplace=True)
else:
    print("Column 'geometry' not found in the DataFrame.")

neighbourhood_data

In [ ]:
# Load the Neighbourhood shapefile
NeighbourhoodBoundary_SHP = os.path.join(folder_path, "Neighbourhoods.shp")
neighbourhood_SHP = gpd.read_file(NeighbourhoodBoundary_SHP)

# Drop all fields except geometry and name fields from neighbourhood_SHP
neighbourhood_SHP = neighbourhood_SHP[['geometry', 'AREA_NA7']]

neighbourhood_SHP

In [ ]:
# Join the neighbourhood dataframe with the Neighbourhood geodataframe using 'NeighbourhoodName' and 'AREA_NA7' respectively
joined_data = neighbourhood_SHP.merge(neighbourhood_data, left_on='AREA_NA7', right_on='NeighbourhoodName', how='right')

joined_data

In [ ]:
# Join the aggregated crime counts with neighborhood joined_data based on the common neighborhood field (NeighbourhoodName)
neighbourhood_merged = joined_data.merge(neighbourhood_counts, on='NeighbourhoodName', how='left')

# Drop rows with missing values (if any)
neighbourhood_merged = neighbourhood_merged.dropna(subset=['Crime_Count'])

In [ ]:
# Check if all geometries are valid
all_valid = neighbourhood_merged.geometry.is_valid.all()

if all_valid:
    print("All geometries are valid.")
else:
    print("Some geometries are invalid.")

In [12]:
# Convert neighbourhood_merged to a geodataframe
neighbourhood_gdf = gpd.GeoDataFrame(neighbourhood_merged, geometry='geometry')

neighbourhood_gdf.info() # Check geo dataframe data type

In [ ]:
# Select relevant features for further analysis
neighbourhood_gdf = neighbourhood_gdf[['NeighbourhoodName', 'Crime_Count', 'Poles_Count', 'POI_Count', 'TCamera_Count', 'TotalPopulation', 'MedianTotalIncome_normalized',
                                      'AverageTotalIncome_normalized','AverageHousePrice_normalized', 'BuildingCoveragePERCENTAGE', 
                                      'ParksOSNAPERCENTAGE', 'BuiltUpPERCENTAGE', 'geometry']]

neighbourhood_gdf

In [ ]:
# Create a spatial weights matrix using queen contiguity
w = weights.Queen.from_dataframe(neighbourhood_gdf)

# Perform Global Moran's I analysis
moran = esda.Moran(neighbourhood_gdf['Crime_Count'], w)

# Print the Moran's I statistic and p-value
print('Moran\'s I:', moran.I)
print('p-value:', moran.p_sim)